# core

> Defines central classes that are re-used throughout the findmycells package (findmycells.core)

- order: 3

In [ ]:
#| default_exp core

In [ ]:
#| export

from abc import ABC, abstractmethod
from findmycells.database import Database
from findmycells.configs import DefaultConfigs, GUIConfigs
from typing import List, Dict, Tuple, Optional, Any
from types import ModuleType
from pathlib import Path, PosixPath
import inspect

In [ ]:
#| hide
from nbdev.showdoc import *

# Handling data processing

The following two classes, `ProcessingObject` and `ProcessingStrategy`, provide the blueprints for all processing strategies and objects that are used throughout the *findmycells* package. As you can see in the corresponding processing step modules (i.e. "preprocess", "segment", or "quantify"), these abstract base classes provide the basic structure of the more specific objects and strategies in each of these modules (e.g. `QuantificationObject` and `QuantificationStrategy` within the "quantify" module inherit from `ProcessingObject` and `ProcessingStrategy`, respectively). While this makes these two classes highly relevant for any developer, regular users of *findmycells* won´t be interacting with them, even if they want to use the API instead of the GUI.

In [ ]:
#| export

class ProcessingObject(ABC):
    
    """
    Abstract base class (inherits from ABC) that defines the general structure of `ProcessingObjects` in findmycells.
    A `ProcessingObject` combines all information needed for the corresponding processing step, 
    i.e. what files are supposed to be processed & how. It also interfaces to the database of the
    project, such that it can automatically update the database with the latest progress.
    """

    @property
    @abstractmethod
    def processing_type(self
                       ) -> str: # string that defines the processing type (e.g. "preprocessing" or "quantification")
        """
        Abstract property. Will be used in the database to keep track of the processing 
        progress of the project. Has to be a string that matches with an available 
        processing module (i.e. "preprocessing", "segmentation", "postprocessing", 
        "quantification").
        """
        pass

    
    @property
    @abstractmethod
    def default_configs(self) -> DefaultConfigs:
        """
        Abstract method that requires its subclasses to define the `default_configs`
        as a property of the class. Thus, this will specify all configuration options
        that come with each subclass, while simultaneously also providing default values
        for each option and, moreover, defining what types of values are allowed for each
        option. Check out the implementation of `DefaultConfigs` in the configs module, or
        have a look at how this is implemented in one of the processing sub-modules, for 
        instance in the "specs.py" file in the preprocessing sub-module.
        """
        pass    
    
    
    @property
    @abstractmethod
    def widget_names(self) -> Dict[str, str]:
        """
        Defines which widgets will be created (e.g. "Checkbox" or "Dropdown"). 
        See `GUIConfigs` to see a full list of available options. 
        """
        pass
    
    
    @property
    @abstractmethod
    def descriptions(self) -> Dict[str, str]:
        """
        Descriptions that will be used as "description" parameter in the created 
        widgets.
        """
        pass
    
    
    @property
    @abstractmethod
    def tooltips(self) -> Optional[Dict[str, str]]:
        """
        Additional information that can be displayed as tooltip by the respective 
        widget. Unfortunately, tooltips are not available for all widget types 
        in the ipywidgets version that is used by *findmycells*.
        """
        return None
    
    
    @abstractmethod
    def _add_processing_specific_infos_to_updates(self, 
                                                 updates: Dict # A dictionary with updates that need to be passed to the database
                                                ) -> Dict: # A dictionary with all updates that need to be passed to the database
        """
        Abstract method that that requires its subclasses to define what updates need to be
        passed to the database, in addition to those that are already covered by the corresponding
        ProcessingStrategies or the "self.update_database()" method. If there are no more 
        information to add, simply return the input 'updates' dictionary without any alterations.
        
        Returns a dictionary with all updates that need to be passed to the database.
        """
        return updates
    
    
    @abstractmethod
    def _processing_specific_preparations(self) -> None:
        """
        Allows to keep the __init__() method clear of any functions, which is 
        required to be able to just instantiate a representative object to create
        its GUI widget (using the GUIConfigs set as attribute). Will be called in
        the prepare_for_processing() method and, thus, still enable processing step
        specific computations before the strategies are executed. 
        
        Leave as "pass" if nothing needs to be done here.
        """
        pass
    
    
    def initialize_gui_configs_and_widget(self) -> None:
        """
        Constructs a `GUIConfigs` from the respectively specified properties 
        and uses the `GUIConfigs.construct_widget` method to build the associated 
        widget and sets it as attribute (self.widget). 
        """
        gui_configs = GUIConfigs(widget_names = self.widget_names,
                                 descriptions = self.descriptions,
                                 tooltips = self.tooltips)
        info_text = ('<div style="font-size: 16px">'
                     '<b>General processing configurations:</b>'
                     '</div>')
        gui_configs.construct_widget(info_text = info_text,
                                     default_configs = self.default_configs)
        setattr(self, 'gui_configs', gui_configs)
        self.widget = self.gui_configs.strategy_widget
    
    
    def export_current_gui_config_values(self) -> Dict:
        """
        Passes the function call further to the `GUIConfigs` object to 
        extract the current configs.
        """
        return self.gui_configs.export_current_config_values()
    
    
    def prepare_for_processing(self,
                               file_ids: List[str], # A list with the file_ids of all files that need to be processed
                               database: Database, # The database of the findmycells project
                              ) -> None:
        self.file_ids = file_ids
        self.database = database
        self._processing_specific_preparations()
    
    
    def run_all_strategies(self, strategies: List, strategy_configs: List[Dict]) -> None:
        """
        Runs all ProcessingStrategies that were passed upon initialization (i.e. self.strategies).
        For this, the corresponding ProcessingStrategy objects will be initialized and their ".run()"
        method will be called, while passing "self" as "processing_object". Finally, it updates the
        database and deletes the ProcessingStrategy object to clear it from memory.
        """
        for strategy, configs in zip(strategies, strategy_configs):
            processing_strategy = strategy()
            self = processing_strategy.run(processing_object = self, strategy_configs = configs)
            self = processing_strategy.update_tracking_histories(processing_object = self, strategy_configs = configs)
            del processing_strategy


    def update_database(self, mark_as_completed: bool=True) -> None:
        """
        For each microscopy file that had to be processed (self.file_ids), the database
        will be updated with the respective processing progress information. Interfaces
        back to the abstract method "self.add_processing_specific_infos_to_updates()" that
        enables the corresponding subclasses to add more specific details before triggering
        the update method of the database.
        """
        for file_id in self.file_ids:
            updates = {}
            if mark_as_completed == True:
                self.database.file_histories[file_id].mark_processing_step_as_completed(processing_step_id = self.processing_type)
            updates = self._add_processing_specific_infos_to_updates(updates = updates)
            self.database.update_file_infos(file_id = file_id, updates = updates)

**Associated public methods:**

In [ ]:
show_doc(ProcessingObject.export_current_gui_config_values)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L131){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.export_current_gui_config_values

>      ProcessingObject.export_current_gui_config_values ()

Passes the function call further to the `GUIConfigs` object to 
extract the current configs.

In [ ]:
show_doc(ProcessingObject.initialize_gui_configs_and_widget)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L113){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.initialize_gui_configs_and_widget

>      ProcessingObject.initialize_gui_configs_and_widget ()

Constructs a `GUIConfigs` from the respectively specified properties 
and uses the `GUIConfigs.construct_widget` method to build the associated 
widget and sets it as attribute (self.widget).

In [ ]:
show_doc(ProcessingObject.prepare_for_processing)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L139){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.prepare_for_processing

>      ProcessingObject.prepare_for_processing (file_ids:List[str],
>                                               database:findmycells.database.Da
>                                               tabase)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| file_ids | typing.List[str] | A list with the file_ids of all files that need to be processed |
| database | Database | The database of the findmycells project |
| **Returns** | **None** |  |

In [ ]:
show_doc(ProcessingObject.run_all_strategies)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L148){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.run_all_strategies

>      ProcessingObject.run_all_strategies (strategies:List,
>                                           strategy_configs:List[Dict])

Runs all ProcessingStrategies that were passed upon initialization (i.e. self.strategies).
For this, the corresponding ProcessingStrategy objects will be initialized and their ".run()"
method will be called, while passing "self" as "processing_object". Finally, it updates the
database and deletes the ProcessingStrategy object to clear it from memory.

In [ ]:
show_doc(ProcessingObject.update_database)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L162){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.update_database

>      ProcessingObject.update_database (mark_as_completed:bool=True)

For each microscopy file that had to be processed (self.file_ids), the database
will be updated with the respective processing progress information. Interfaces
back to the abstract method "self.add_processing_specific_infos_to_updates()" that
enables the corresponding subclasses to add more specific details before triggering
the update method of the database.

**For developers:**

Associated properties that need to be implemented in subclasses inheriting from `ProcessingObject`. Feel free to check out the available implementations to see how this should look like, for instance in the "findmycells.preprocessing.specs" or "findmycells.segmentation.specs" submodules.

In [ ]:
show_doc(ProcessingObject.default_configs)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L40){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.default_configs

>      ProcessingObject.default_configs ()

Abstract method that requires its subclasses to define the `default_configs`
as a property of the class. Thus, this will specify all configuration options
that come with each subclass, while simultaneously also providing default values
for each option and, moreover, defining what types of values are allowed for each
option. Check out the implementation of `DefaultConfigs` in the configs module, or
have a look at how this is implemented in one of the processing sub-modules, for 
instance in the "specs.py" file in the preprocessing sub-module.

In [ ]:
show_doc(ProcessingObject.descriptions)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L65){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.descriptions

>      ProcessingObject.descriptions ()

Descriptions that will be used as "description" parameter in the created 
widgets.

In [ ]:
show_doc(ProcessingObject.processing_type)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L27){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.processing_type

>      ProcessingObject.processing_type ()

Abstract property. Will be used in the database to keep track of the processing 
progress of the project. Has to be a string that matches with an available 
processing module (i.e. "preprocessing", "segmentation", "postprocessing", 
"quantification").

In [ ]:
show_doc(ProcessingObject.tooltips)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L75){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.tooltips

>      ProcessingObject.tooltips ()

Additional information that can be displayed as tooltip by the respective 
widget. Unfortunately, tooltips are not available for all widget types 
in the ipywidgets version that is used by *findmycells*.

In [ ]:
show_doc(ProcessingObject.widget_names)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L55){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject.widget_names

>      ProcessingObject.widget_names ()

Defines which widgets will be created (e.g. "Checkbox" or "Dropdown"). 
See `GUIConfigs` to see a full list of available options.

Associated abstract methods that need to be implemented in subclasses inheriting from `ProcessingObject`:

In [ ]:
show_doc(ProcessingObject._add_processing_specific_infos_to_updates)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L85){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject._add_processing_specific_infos_to_updates

>      ProcessingObject._add_processing_specific_infos_to_updates (updates:Dict)

Abstract method that that requires its subclasses to define what updates need to be
passed to the database, in addition to those that are already covered by the corresponding
ProcessingStrategies or the "self.update_database()" method. If there are no more 
information to add, simply return the input 'updates' dictionary without any alterations.

Returns a dictionary with all updates that need to be passed to the database.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| updates | typing.Dict | A dictionary with updates that need to be passed to the database |
| **Returns** | **typing.Dict** | **A dictionary with all updates that need to be passed to the database** |

In [ ]:
show_doc(ProcessingObject._processing_specific_preparations)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L100){target="_blank" style="float:right; font-size:smaller"}

### ProcessingObject._processing_specific_preparations

>      ProcessingObject._processing_specific_preparations ()

Allows to keep the __init__() method clear of any functions, which is 
required to be able to just instantiate a representative object to create
its GUI widget (using the GUIConfigs set as attribute). Will be called in
the prepare_for_processing() method and, thus, still enable processing step
specific computations before the strategies are executed. 

Leave as "pass" if nothing needs to be done here.

<br>
<br>
<br>

In [ ]:
#| export

class ProcessingStrategy(ABC):
    
    """
    Abstract base class that defines the general structure of `ProcessingStrategies` in findmycells.
    A `ProcessingStrategy` combines all functions that are required for one particular processing step, 
    e.g. `ConvertTo8Bit` is a `ProcessingStrategy` in the "preprocess" module and converts the corresponding
    images into 8-bit.
    """

    @property
    @abstractmethod
    def processing_type(self):
        """
        Abstract property. Will be used for instance in the database to keep track of the processing 
        progress of the project. Has to be a string that matches with an available 
        processing module (i.e. "preprocessing", "segmentation", "postprocessing", 
        "quantification").
        """
        pass
    
    
    @property
    @abstractmethod
    def default_configs(self) -> DefaultConfigs:
        """
        Abstract method that requires its subclasses to define the `default_configs`
        as a property of the class. Thus, this will specify all configuration options
        that come with each subclass, while simultaneously also providing default values
        for each option and, moreover, defining what types of values are allowed for each
        option. Check out the implementation of `DefaultConfigs` in the configs module, or
        have a look at how this is implemented in one of the processing sub-modules, for 
        instance in the "specs.py" file in the preprocessing sub-module.
        """
        pass
    
    
    @property
    @abstractmethod
    def widget_names(self) -> Dict[str, str]:
        """
        Defines which widgets will be created (e.g. "Checkbox" or "Dropdown"). 
        See `GUIConfigs` to see a full list of available options. 
        """
        pass
    
    
    @property
    @abstractmethod
    def descriptions(self) -> Dict[str, str]:
        """
        Descriptions that will be used as "description" parameter in the created 
        widgets.
        """
        pass
    
    
    @property
    @abstractmethod
    def tooltips(self) -> Optional[Dict[str, str]]:
        """
        Additional information that can be displayed as tooltip by the respective 
        widget. Unfortunately, tooltips are not available for all widget types 
        in the ipywidgets version that is used by *findmycells*.
        """
        pass
    
    
    @property
    @abstractmethod
    def dropdown_option_value_for_gui(self) -> str:
        """
        This string will be used as the option in the strategy selection 
        dropdown in the GUI of *findmycells*.
        """
        pass


    @abstractmethod
    def run(self, processing_object: ProcessingObject, strategy_configs: Dict) -> ProcessingObject:
        """
        Here comes the code that actually does the processing of the data.
        """
        return processing_object

    
    @abstractmethod
    def _add_strategy_specific_infos_to_updates(self, updates: Dict) -> Dict:
        """
        Allows to add processing specific information that is deemed relevant 
        to the `FileHistory` objects in the `Database`. If there are no relevant 
        information, simply return the "updates" dictionary again without changes.
        """
        return updates


    @property
    def strategy_name(self):
        return self.__class__.__name__ 

    
    def initialize_gui_configs_and_widget(self) -> None:
        """
        Constructs a `GUIConfigs` from the respectively specified properties 
        and uses the `GUIConfigs.construct_widget` method to build the associated 
        widget and sets it as attribute (self.widget). 
        """
        gui_configs = GUIConfigs(widget_names = self.widget_names,
                                 descriptions = self.descriptions,
                                 tooltips = self.tooltips)
        docstring_in_html_syntax = self._convert_docstring_to_html_syntax()
        gui_configs.construct_widget(info_text = docstring_in_html_syntax,
                                     default_configs = self.default_configs)
        setattr(self, 'gui_configs', gui_configs)
        self.widget = self.gui_configs.strategy_widget
    
    
    def export_current_gui_config_values(self) -> Dict:
        """
        Passes the function call further to the `GUIConfigs` object to 
        extract the current configs.
        """
        return self.gui_configs.export_current_config_values()
    
    
    def update_tracking_histories(self, processing_object: ProcessingObject, strategy_configs: Dict) -> ProcessingObject:
        for file_id in processing_object.file_ids:
            strategy_configs_with_updates = self._add_strategy_specific_infos_to_updates(updates = strategy_configs)
            tracking_history = processing_object.database.file_histories[file_id]
            tracking_history.track_processing_strat(processing_step_id = self.processing_type,
                                                    processing_strategy_name = self.strategy_name,
                                                    strategy_configs = strategy_configs_with_updates)
        return processing_object


    def _convert_docstring_to_html_syntax(self) -> str:
        """
        To visualize the description of each strategy in a proper way in the GUI, the online hosted
        documentation, and in the docstrings, we added some custom "string commands". These allow us
        to achieve some HTML formatting (required for both GUI and the documentation), while they
        don't interfere with the layout of the docstring. For this, the follwing keys were introduced:
        
        '\b': Every '\b' in the original docstring will be replaced by a '<br>' upon conversion of the
        docstring into HTML, creating a linebreak. It will simply be ignored in the original docstring
        and is therefore not visible.
        
        '\1' and '\2': To make indentation blocks possible, '\1' and '\2' were introduced. '\1' denotes
        the beginning of an indentation block, whereas '\2' marks its end. Accordingly, '\1' will be 
        converted into '<div style="margin-left: 2em;">', and '\2' will be converted into '</div>'.
        They can also be used to stack indentations, simply by using multiple '\1's before closing
        one after the other by adding correspondingly matching '\2's. Again, they will be ignored in
        the original docstring and therefore not be visible.
        """
        docstring = self.__doc__
        if docstring == None:
            docstring = ''
        partially_converted_docstring = docstring.replace('\1', '<div style="margin-left: 2em;">')
        partially_converted_docstring = partially_converted_docstring.replace('\2', '</div>')
        partially_converted_docstring = partially_converted_docstring.replace('\b', '<br>')
        html_converted_docstring = partially_converted_docstring.replace('  ', '')
        return html_converted_docstring

**Associated public methods:**

In [ ]:
show_doc(ProcessingStrategy.export_current_gui_config_values)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L294){target="_blank" style="float:right; font-size:smaller"}

### ProcessingStrategy.export_current_gui_config_values

>      ProcessingStrategy.export_current_gui_config_values ()

Passes the function call further to the `GUIConfigs` object to 
extract the current configs.

In [ ]:
show_doc(ProcessingStrategy.initialize_gui_configs_and_widget)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L278){target="_blank" style="float:right; font-size:smaller"}

### ProcessingStrategy.initialize_gui_configs_and_widget

>      ProcessingStrategy.initialize_gui_configs_and_widget ()

Constructs a `GUIConfigs` from the respectively specified properties 
and uses the `GUIConfigs.construct_widget` method to build the associated 
widget and sets it as attribute (self.widget).

In [ ]:
show_doc(ProcessingStrategy.update_tracking_histories)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L302){target="_blank" style="float:right; font-size:smaller"}

### ProcessingStrategy.update_tracking_histories

>      ProcessingStrategy.update_tracking_histories
>                                                    (processing_object:__main__
>                                                    .ProcessingObject,
>                                                    strategy_configs:Dict)

**For developers:**

Associated properties that need to be implemented in subclasses inheriting from `ProcessingStrategy`. Feel free to check out the available implementations to see how this should look like, for instance in the "findmycells.preprocessing.specs" & "findmycells.preprocessing.strategies" submodules.

In [ ]:
show_doc(ProcessingStrategy.default_configs)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L201){target="_blank" style="float:right; font-size:smaller"}

### ProcessingStrategy.default_configs

>      ProcessingStrategy.default_configs ()

Abstract method that requires its subclasses to define the `default_configs`
as a property of the class. Thus, this will specify all configuration options
that come with each subclass, while simultaneously also providing default values
for each option and, moreover, defining what types of values are allowed for each
option. Check out the implementation of `DefaultConfigs` in the configs module, or
have a look at how this is implemented in one of the processing sub-modules, for 
instance in the "specs.py" file in the preprocessing sub-module.

In [ ]:
show_doc(ProcessingStrategy.descriptions)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L226){target="_blank" style="float:right; font-size:smaller"}

### ProcessingStrategy.descriptions

>      ProcessingStrategy.descriptions ()

Descriptions that will be used as "description" parameter in the created 
widgets.

In [ ]:
show_doc(ProcessingStrategy.processing_type)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L189){target="_blank" style="float:right; font-size:smaller"}

### ProcessingStrategy.processing_type

>      ProcessingStrategy.processing_type ()

Abstract property. Will be used for instance in the database to keep track of the processing 
progress of the project. Has to be a string that matches with an available 
processing module (i.e. "preprocessing", "segmentation", "postprocessing", 
"quantification").

In [ ]:
show_doc(ProcessingStrategy.tooltips)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L236){target="_blank" style="float:right; font-size:smaller"}

### ProcessingStrategy.tooltips

>      ProcessingStrategy.tooltips ()

Additional information that can be displayed as tooltip by the respective 
widget. Unfortunately, tooltips are not available for all widget types 
in the ipywidgets version that is used by *findmycells*.

In [ ]:
show_doc(ProcessingStrategy.widget_names)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L216){target="_blank" style="float:right; font-size:smaller"}

### ProcessingStrategy.widget_names

>      ProcessingStrategy.widget_names ()

Defines which widgets will be created (e.g. "Checkbox" or "Dropdown"). 
See `GUIConfigs` to see a full list of available options.

In [ ]:
show_doc(ProcessingStrategy.dropdown_option_value_for_gui)  

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L247){target="_blank" style="float:right; font-size:smaller"}

### ProcessingStrategy.dropdown_option_value_for_gui

>      ProcessingStrategy.dropdown_option_value_for_gui ()

This string will be used as the option in the strategy selection 
dropdown in the GUI of *findmycells*.

Associated abstract methods that need to be implemented in subclasses inheriting from `ProcessingStrategy`:

In [ ]:
show_doc(ProcessingStrategy.run)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L256){target="_blank" style="float:right; font-size:smaller"}

### ProcessingStrategy.run

>      ProcessingStrategy.run (processing_object:__main__.ProcessingObject,
>                              strategy_configs:Dict)

Here comes the code that actually does the processing of the data.

In [ ]:
show_doc(ProcessingStrategy._add_strategy_specific_infos_to_updates)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L264){target="_blank" style="float:right; font-size:smaller"}

### ProcessingStrategy._add_strategy_specific_infos_to_updates

>      ProcessingStrategy._add_strategy_specific_infos_to_updates (updates:Dict)

Allows to add processing specific information that is deemed relevant 
to the `FileHistory` objects in the `Database`. If there are no relevant 
information, simply return the "updates" dictionary again without changes.

<br>
<br>
<br>

# Handling data import

Furthermore, the following two classes `DataLoader` and `DataReader` will be re-used throughout the *findmycells* package to load data into your *findmycells* project. You can find the specific implementations of `DataReader`s in the findmycells.reader module.

In [ ]:
#| export

class DataReader(ABC):
    
    """
    Abstract base class that defines the general structure of DataReader subclasses.
    Essentially, it demands the corresponding subclasses to define the "readable_filetype_extensions" 
    attribut, as well as the "set_optional_configs()" and the "read()" methods.
    """
    
    @property
    @abstractmethod
    def readable_filetype_extensions(self) -> List[str]:
        """
        Property that will denote which filetype extensions the respective DataReader subclass can handle.
        """
        pass
    
    
    @abstractmethod
    def read(self, filepath: Path, reader_configs: Dict) -> Any:
        """
        This method eventually reads the data stored at the given filepath applying the specified configs.
        The returned datatype will be different for each DataReader subclass, e.g. a numpy array of a specific
        shape for MicroscopyImageReaders, or a shapely Polygon for ROIReaders.
        """
        pass
    
    
    @abstractmethod
    def assert_correct_output_format(self, output: Any) -> None:
        """
        Run an assert to validate that the data was actually read in the correct way and that the created output
        matches the intended format!
        """
        pass

**Associated public methods (all abstract methods, that need to be implemented):**

In [ ]:
show_doc(DataReader.read)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L358){target="_blank" style="float:right; font-size:smaller"}

### DataReader.read

>      DataReader.read (filepath:pathlib.Path, reader_configs:Dict)

This method eventually reads the data stored at the given filepath applying the specified configs.
The returned datatype will be different for each DataReader subclass, e.g. a numpy array of a specific
shape for MicroscopyImageReaders, or a shapely Polygon for ROIReaders.

In [ ]:
show_doc(DataReader.assert_correct_output_format)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L368){target="_blank" style="float:right; font-size:smaller"}

### DataReader.assert_correct_output_format

>      DataReader.assert_correct_output_format (output:Any)

Run an assert to validate that the data was actually read in the correct way and that the created output
matches the intended format!

<br>
<br>
<br>

In [ ]:
#| export

class DataLoader:
    
    """
    Works as an interface between whatever needs to import data (e.g. a `PreprocessingObject`) 
    and the `DataReader`s. It will look for an adequate reader implemented for the specific 
    data type (i.e. filetype) at hand.
    """
    
    def determine_reader(self, file_extension: str, data_reader_module: ModuleType) -> DataReader:
        """
        Check whether there is a reader implemented in the requested reader submodule that 
        can handle the specified filetype inferred from its extension.
        """
        available_reader = None
        for name, data_reader in inspect.getmembers(data_reader_module):
            if (name.endswith('Reader') == True) & (name != 'DataReader'):
                if file_extension in data_reader().readable_filetype_extensions:
                    available_reader = data_reader
        if available_reader == None:
            raise NotImplementedError(f'Unfortunately, there is no DataReader implemented in {data_reader_module} '
                                      f'which can handle your filetype ("{file_extension}").')
        return available_reader
    
    
    def load(self, data_reader_class: DataReader, filepath: PosixPath, reader_configs: Dict) -> Any:
        """
        Uses the provided `DataReader` subclass to import the data.
        """
        data_reader = data_reader_class()
        # data_reader.set_optional_configs(database = database)
        data = data_reader.read(filepath = filepath, reader_configs = reader_configs)
        data_reader.assert_correct_output_format(output = data)
        return data                 

**Associated public methods:**

In [ ]:
show_doc(DataLoader.determine_reader)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L384){target="_blank" style="float:right; font-size:smaller"}

### DataLoader.determine_reader

>      DataLoader.determine_reader (file_extension:str,
>                                   data_reader_module:module)

Check whether there is a reader implemented in the requested reader submodule that 
can handle the specified filetype inferred from its extension.

In [ ]:
show_doc(DataLoader.load)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/core.py#L400){target="_blank" style="float:right; font-size:smaller"}

### DataLoader.load

>      DataLoader.load (data_reader_class:__main__.DataReader,
>                       filepath:pathlib.PosixPath, reader_configs:Dict)

Uses the provided `DataReader` subclass to import the data.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()